In [14]:
#Sklearn

from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.utils import resample,shuffle
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.metrics import make_scorer, precision_score, recall_score,f1_score, accuracy_score,classification_report,confusion_matrix
from sklearn.metrics import make_scorer,precision_score,recall_score,accuracy_score,f1_score

#NLTK
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from IPython.display import display, HTML
#Other
import pandas as pd
import re
import string
import matplotlib.pyplot as pypl
import numpy as np
import seaborn as sd
#import imblearn
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler

In [15]:
contract = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [16]:
stopword = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()


# boogiepop_file = 'D:\Github Projects\Heriot-Watt-Msc-Project-Sentiment-Analysis\Data Cleaner\Boogiepop cleaned\Cleaned_Boogiepop_Wa_Waranai_Episode_' + str(ep) + '_Comment_list.csv'
# kaguya_file = 'D:\Github Projects\Heriot-Watt-Msc-Project-Sentiment-Analysis\Data Cleaner\Kaguya-sama cleaned\Cleaned_Kaguya_sama_Episode_' + str(1) + '_Comment_list.csv'
# slime_file = 'D:\Github Projects\Heriot-Watt-Msc-Project-Sentiment-Analysis\Data Cleaner\Tensei Slime cleaned\Cleaned_Tensei_Slime_Episode_' + str(ep) + '_Comment_list.csv'


def penntag(pen):
    morphy_tag = {'NN': 'n', 'JJ': 'a',
                  'VB': 'v', 'RB': 'r'}
    try:
        return morphy_tag[pen[:2]]
    except:
        return 'n'


def comment_cleaner(comm, comment_array):
    temp_comm = []
    megos = ' '
    uncontracted = ' '.join([contract[word] if word in contract else word for word in comm.lower().split()])
    stopwords_removed = [word for word in uncontracted.lower().split() if word not in stopword]
    POS_words = nltk.pos_tag(stopwords_removed)
    for i in range(0, len(POS_words)):
        lemmas = lemma.lemmatize(POS_words[i][0], pos=penntag(POS_words[i][1]))
        temp_comm.append(lemmas)
    # print(temp_comm)
    megos = ' '.join(word for word in temp_comm)
    #comment_array.append(megos)
    return megos
   # comment_array.clear()

file_1 = 15
file_2 = 16
file_3 = 17
test_no = 18

for ep in range(1, 2):
    pd.set_option('display.max_colwidth', -1)
    
    train_array = []
    test_array = []
    train_target = []
    comtest_array = []
    kaguya_file1 = 'D:\Github Projects\Heriot-Watt-Msc-Project-Sentiment-Analysis\Manually determines\Boogiepop cleaned\Boogiepop Wa Waranai Episode ' + str(file_1) + ' .csv'
    df1 = pd.read_csv(kaguya_file1, encoding='utf-8-sig')
    
    kaguya_file2 = 'D:\Github Projects\Heriot-Watt-Msc-Project-Sentiment-Analysis\Manually determines\Boogiepop cleaned\Boogiepop Wa Waranai Episode ' + str(file_2) + ' .csv'

    df2 = pd.read_csv(kaguya_file2, encoding='utf-8-sig')
    
    kaguya_file3 = 'D:\Github Projects\Heriot-Watt-Msc-Project-Sentiment-Analysis\Manually determines\Boogiepop cleaned\Boogiepop Wa Waranai Episode ' + str(file_3) + ' .csv'
    df3 = pd.read_csv(kaguya_file3, encoding='utf-8-sig')
    

    
    df12 = pd.concat([df1,df2,df3])
    print(df12['Comments'])
    df12 = df12[['Comments','Actual Polarity']]
    df12['Comments'] = df12['Comments'].astype(str)
    
    df12['Comments'] = df12['Comments'].astype(str).apply(lambda s: comment_cleaner(s, train_array))

    # Remove punctuation marks and tokenize each and every word
    df12['Comments'] = df12['Comments'].astype(str).str.replace('[^\w\s]', ' ')
    df12['Comments'] = df12['Comments'].astype(str).str.replace('[\d+]', ' ')
    df12['Comments'] = df12['Comments'].astype(str).str.replace('(^| ).(( ).)*( |$)', ' ')

    
    # Split into positive and negative datasets
    pos_df = df12[df12['Actual Polarity'] == 1]
    neg_df = df12[df12['Actual Polarity'] == 0]
    neu_df = df12[df12['Actual Polarity'] == 2]
    
    df_len = len(pos_df)
  
    train_df = pd.concat([pos_df, neg_df,neu_df])
    train_df = train_df.reset_index(drop=True)
   

    train_array = []
    test_array = []
    train_target = []
    comtest_array = []
    y = train_df['Actual Polarity']

    x_train, x_test, y_train, y_test = train_test_split(train_df['Comments'], train_df['Actual Polarity'], test_size=0.2,random_state=22)

    vec = TfidfVectorizer(ngram_range=(1, 2),max_features=20000,sublinear_tf=True)
    x_tr = vec.fit_transform(x_train)
    sm = RandomOverSampler(random_state=77)

    X_train_res, y_train_res = sm.fit_sample(x_tr, y_train)
   
    NB = MultinomialNB()
    NB.fit(X_train_res,y_train_res)
    
    
    test_file = pd.read_csv(
        'D:\Github Projects\Heriot-Watt-Msc-Project-Sentiment-Analysis\Manually determines\Boogiepop cleaned\Boogiepop Wa Waranai Episode ' + str(test_no) + ' .csv', index_col=0, encoding='utf-8-sig')
    test_file['Comments'] = test_file['Comments'].astype(str)
    
    poso = test_file[test_file['Polarity'] == 1]
    nego = test_file[test_file['Polarity'] == 0]
    neuo = test_file[test_file['Polarity'] == 2]
    
    aeuo = pd.concat([poso,nego,neuo])

    test_file['Comments'] = test_file['Comments'].astype(str).apply(lambda s: comment_cleaner(s, test_array))

    # Remove punctuation marks and tokenize each and every word
    test_file['Comments'] = test_file['Comments'].astype(str).str.replace('[^\w\s]', ' ')
    test_file['Comments'] = test_file['Comments'].astype(str).str.replace('[\d+]', ' ')
    test_file['Comments'] = test_file['Comments'].astype(str).str.replace('(^| ).(( ).)*( |$)', ' ')

    pos_test_file = test_file[test_file['Polarity'] == 1]
    neg_test_file = test_file[test_file['Polarity'] == 0]
    neu_test_file = test_file[test_file['Polarity'] == 2]

    train_test = pd.concat([pos_test_file, neg_test_file,neu_test_file])
    train_test = train_test.reset_index(drop=True)
    
    xe = vec.transform(test_file['Comments'])
    x_pred = NB.predict(xe)
   
    da = {'Comments': aeuo['Comments'],'Polarity':test_file['Polarity'],'Actual Polarity':x_pred}
    xool = pd.DataFrame(da)
    print(xool)
    ars = display(HTML(xool.to_html(index=False)))
    xool.to_csv('D:\Github Projects\Heriot-Watt-Msc-Project-Sentiment-Analysis\Manually determines\Boogiepop cleaned\Boogiepop Wa Waranai Episode ' + str(test_no) + ' .csv', index=False, encoding='utf-8-sig')

0      Yorokobe, Shounen. will finally come true.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

Comments,Polarity,Actual Polarity
"This show does have some pretty good quotes, occasionally. That's one I'm sure to remember.",1,1
"“I order to have the right to asked to be killed, you have to actually live first” I’ve been goin thru a rough time lately. this line hit me in the chest, though. Boogiepop is my favorite character in a while because they drop this knowledge that I really need to hear sometimes.",1,0
"Yeah, it has quite a lot of good quotes, hope they get compiled somewhere after it ends.",1,1
"I would also toss Welcome to the NHK in that list. I watched that series at a time when I was extremely depressed and it gave me some comfort in an ""I'm not alone"" way",1,0
"Shit, I completely forgot about that show. Well, now I have something new to start",1,1
"I've been meaning to start that series but I always forget when I'm scrolling through searching for something to watch. I think I'll try and start it today, thanks for the reminder!",1,1
Yeah there are some very memorable lines in this series.,1,1
"that's an interesting theory and one i hadn't thought of, wonder if it's true.",1,1
"I think the kid just got next level validated by the guy. Imagine you're some loser nerd and then Bill Gates dies leaving behind this weird tower. It's this huge attraction and everyone goes inside. Everyone falls asleep, and then your friend wakes you up but has no clue about anything. You then serve as the intellectual backbone of the investigation, such as finding hollow places in walls etc... Suddenly you get to the control room and Bill Gates lets you in on this huge conspiracy and declares you to be the chosen one simply due to the fact that you managed to find this room, even if you don't know it yet/nothing in your life proves it yet. My man just got Isekai-ed IRL. Like there's no way you don't 100% believe Bill Gates at that moment right? Now I think the King of Distortion is his own entity and just bandwagoned onto Anime Bill Gates' plan to use it for his own purposes. Which i think he wants to king midas people by fixing their emotional problems",1,2
"Well the archer boy already had a honor of shooting Manticore through the head, it's not really out of question for him to imagine himself as a crime-fighting superhero. But he actually looks kinda humble to be honest.",1,1
